# Libraries

In [3]:
import pandas as pd

In [ ]:
# explore langchain, langgraph, llamaindex, llamagraph?
# crewAI, AG2
# neo4j

# Data Loading

# Data Ingestion

## Text Parsing

## Tabular Data Parsing (if exists)

- use a Text-to-SQL engine

## Audio Parsing

## Image Parsing

## Video Parsing

## Embeddings + Vector Database

## Graph Construction/Generation

# Model Loading

# Model + MM-RAG Experimentation

In [ ]:
# sources must be cited



## Traditional/Static RAG


In [ ]:
# w/ reranking


## Agentic Multimodal RAG

## Multimodal RAG with Reasoning

## Agentic Multimodal RAG with Reasoning

# Benchmarking Model

- use benchmarks online and compare the architectures
- compare per model inference time, quality, etc


ex. 

- BEIR
- MS MARCO
- TREC
- MuSiQue
- 2WikiMultihopQA
- AgentG
- HotpotQA
- RAGBench
- BERGEN
- FlashRAG Toolkit
- GNN-RAG


multimodal RAG benchmarks?
- MRAG-Bench
- MRAMG-Bench
- zhzhen23/DynVQA
- https://github.com/JarvisUSTC/Awesome-Multimodal-RAG

create own benchmark based on own data
- https://huggingface.co/learn/cookbook/en/rag_evaluation

# App Frontend

# Model Serving

In [ ]:
#